In [38]:
from darts import TimeSeries
from darts.models import NBEATSModel, NHiTSModel, TransformerModel, TSMixerModel
from darts.utils.losses import *
from darts.metrics import metrics as darts_metrics
from utils import data_handling, helpers
import torch
import numpy as np
import os
import pickle
import pandas as pd
import config
import copy

# Constants
DEVICE = [1]
NUM_EPOCHS = 5
IN_LEN = 96
OUT_LEN = 96
LOSS_FN = torch.nn.MSELoss()
LAYER_WIDTH = 256
NUM_STACKS = 4
NUM_BLOCKS = 2
NUM_LAYERS = 2
COEFFS_DIM = 5
DROPOUT = 0.2
VERBOSE = False
TRAIN_EPOCHS = 10
TUNE_EPOCHS = 5
four_weeks = -24*7*4

metrics_output_path = config.CONFIG_OUTPUT_PATH["darts"] / "darts_tl_metrics.csv"

# Helper function for model checkpointing
def save_checkpoint(model, filename):
    torch.save(model.state_dict(), filename)

def load_checkpoint(model, filename):
    model.load_state_dict(torch.load(filename))
    model.eval()

In [35]:
def extend_source_to_target_id_count(source, target):
    source_id_count = source["train"].shape[1]
    target_id_count = target["train"].shape[1]

    full_repeats = target_id_count // source_id_count
    remainder = target_id_count % source_id_count

    repeated_tensor = source["train"].repeat(1, full_repeats)
    remainder_tensor = source["train"][:, :remainder]
    source_train = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    
    assert target_id_count == source_train.size(1), f"Reshaping was incorrect. Target_train = {target_id_count}, source_train = {source_train.size(1)}."

    repeated_tensor = source["validation"].repeat(1, full_repeats)
    remainder_tensor = source["validation"][:, :remainder]
    source_validation = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    assert target_id_count == source_validation.size(1), f"Reshaping was incorrect. Target_val = {target_id_count}, source_val = {source_validation.size(1)}."

    return source_train, source_validation


def process_tl_data(source_data, target_data):
    # either reshape source or target dataset according to which has less IDs
    source_ids = source_data["train"].size(1)
    target_ids = target_data["test"].size(1)

    fine_tune_horizon = -24*7*4
    target_test = target_data["test"]
    target_fine_tuning = target_data["train"][fine_tune_horizon:,:]

    # remove IDs if source is bigger than target or
    # repeat IDs if target is bigger than source
    if target_ids < source_ids:
        source_train = source_data["train"][:,:target_ids]
        source_validation = source_data["validation"][:,:target_ids]
    else:
        source_train, source_validation = extend_source_to_target_id_count(source_data, target_data)

    # convert to TimeSeries dataframe
    source_train = TimeSeries.from_values(source_train)
    source_validation = TimeSeries.from_values(source_validation)
    target_test = TimeSeries.from_values(target_test)
    target_fine_tuning = TimeSeries.from_values(target_fine_tuning)
    target_train = TimeSeries.from_values(target_data["train"])

    tl_dataset = {
                    "source_train" : source_train,
                    "source_validation" : source_validation,
                    "target_fine_tuning" : target_fine_tuning,
                    "target_test" : target_test,
                    "target_train" : target_train
                }

    return tl_dataset

In [22]:
def train_nhits(ts_train, ts_val, epochs=1):
    nhits_model = NHiTSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        activation='ReLU',
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        dropout=DROPOUT,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "gpu",  "devices" : DEVICE},
    )
    nhits_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return nhits_model

# Train NBEATS model
def train_nbeats(ts_train, ts_val, epochs=1):
    nbeats_model = NBEATSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        batch_size=32,
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        expansion_coefficient_dim=COEFFS_DIM,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        activation='ReLU',
        optimizer_kwargs={"lr": 0.0005},
        pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "gpu",  "devices" : DEVICE},
    )
    nbeats_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return nbeats_model

# Train Transformer model
def train_transformer(ts_train, ts_val, epochs=1):
    transformer_model = TransformerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN,
        d_model=LAYER_WIDTH, 
        nhead=4, 
        num_encoder_layers=3, 
        num_decoder_layers=3, 
        dim_feedforward=LAYER_WIDTH, 
        dropout=DROPOUT, 
        activation='relu', 
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "gpu",  "devices" : DEVICE},

    )
    transformer_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return transformer_model

# Train TSMixer model
def train_tsmixer(ts_train, ts_val, epochs=1):
    tsmixer_model = TSMixerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN, 
        hidden_size=LAYER_WIDTH, 
        ff_size=LAYER_WIDTH, 
        num_blocks=NUM_BLOCKS, 
        activation='ReLU', 
        dropout=DROPOUT, 
        loss_fn=LOSS_FN,
        norm_type='LayerNorm', 
        use_reversible_instance_norm=True,
        pl_trainer_kwargs={"enable_progress_bar": True, "accelerator": "gpu", "devices" : DEVICE},

    )
    tsmixer_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return tsmixer_model

In [31]:
def evaluate(model, target_test):
    """
    Evaluates models on target test set
    Input:  -trained model
            -List of target test sets shaped according to models

    Output: Dict{MSE, MAE}
    """

    forecasting_endpoint = int(len(target_test)) - 96*2
    window = [target_test[i:i+96] for i in range(0, forecasting_endpoint, 5)]
    target = [target_test[i+96:i+96+96] for i in range(0, forecasting_endpoint, 5)]

    # predict over dataloader with slidingwindow implementation and 5 time step shifts for each input
    predictions = model.predict(n=96, series=window)

    mse = darts_metrics.mse(predictions, target)
    mae = darts_metrics.mae(predictions, target)

    mse = sum(mse) / len(predictions)
    mae = sum(mae) / len(predictions)

    return {'MSE': mse, 'MAE': mae}


def fine_tune_model(model, target_fine_tuning, epochs=1):
    """
    Fine tune models over specified epochs

    Input:  -trained models
            -fine tuning dataset
            -epochs

    Returns: fitted models
    """

    model.fit(
            target_fine_tuning,
            num_loader_workers=4,
            epochs=epochs,
            max_samples_per_ts=None,
        )
 
    return model

In [11]:
# use electricity dataset
electricity_dict = data_handling.format_electricity()

for key, value in electricity_dict.items():
			electricity_dict[key]= data_handling.df_to_tensor(value)

# normalize train and use matrics for val and test
electricity_dict["4_weeks_train"] = electricity_dict["train"][four_weeks:,:]
electricity_dict["train"], train_standardize_dict = helpers.custom_standardizer(electricity_dict["train"])
electricity_dict["validation"], _ = helpers.custom_standardizer(electricity_dict["validation"], train_standardize_dict)
electricity_dict["test"], _ = helpers.custom_standardizer(electricity_dict["test"], train_standardize_dict)
electricity_dict["4_weeks_train"], _ = helpers.custom_standardizer(electricity_dict["4_weeks_train"], train_standardize_dict)

# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
bavaria_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
bavaria_dict["4_weeks_train"] = bavaria_dict["train"][four_weeks:,:]

# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
gp_dict["4_weeks_train"] = gp_dict["train"][four_weeks:,:]

Length train set: 209 days, 0:00:00
Length validation set: 34 days, 0:00:00
Saving train, validation and test df for faster loading


In [12]:
tl_setups = {
    "elec_to_bavaria" : (electricity_dict, bavaria_dict), 
    "elec_to_gp" : (electricity_dict, gp_dict),
    "bavaria_to_elec" : (bavaria_dict, electricity_dict), 
    "bavaria_to_gp" : (bavaria_dict, gp_dict), 
    "gp_to_bavaria": (gp_dict, bavaria_dict), 
    "gp_to_elec" : (gp_dict, electricity_dict)
     }

In [37]:
metrics = ["MSE", "MAE"]
fine_tuning_scenarios = ["Zero-Shot", "four_weeks", "full"]
index = pd.MultiIndex.from_product([tl_setups.keys(), fine_tuning_scenarios, metrics], names=["Setup", "Fine-Tuning", "Metric"])
tmp = pd.DataFrame(columns=models.keys(), index=index)

tmp

NHiTS NBEATS Transformer TSMixer
Setup           Fine-Tuning Metric                                 
elec_to_bavaria Zero-Shot   MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                four_weeks  MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                full        MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
elec_to_gp      Zero-Shot   MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                four_weeks  MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                full        MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
bavaria_to_elec Zero-Shot   MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                four_weeks  MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                full        MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
bavaria_to_gp   Zero-Shot   MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                four_weeks  MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                full        MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
gp_to_bavaria   Zero-Shot   MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                four_weeks  MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                full        MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
gp_to_elec      Zero-Shot   MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                four_weeks  MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN
                full        MSE      NaN    NaN         NaN     NaN
                            MAE      NaN    NaN         NaN     NaN

In [36]:
TRAIN_EPOCHS = 10
TUNE_EPOCHS = 5


# Initialize the DataFrame
models = {
    "NHiTS": train_nhits, 
    "NBEATS": train_nbeats, 
    "Transformer": train_transformer, 
    "TSMixer": train_tsmixer
}

try:
    results_df = pd.read_csv(metrics_output_path)
except FileNotFoundError:
    metrics = ["MSE", "MAE"]
    fine_tuning_scenarios = ["Zero-Shot", "four_weeks", "full"]
    index = pd.MultiIndex.from_product([tl_setups.keys(), fine_tuning_scenarios, metrics], names=["Setup", "Fine-Tuning", "Metric"])
    results_df = pd.DataFrame(columns=models.keys(), index=index)

# Helper functions
def update_metrics(setup_name, model_name, fine_tuning, mae, mse):
    results_df.loc[(setup_name, "MAE", fine_tuning), model_name] = mae
    results_df.loc[(setup_name, "MSE", fine_tuning), model_name] = mse

def is_metric_filled(setup_name, model_name, fine_tuning):
    # Check if specific metrics for a model in a setup and fine-tuning scenario are NaN or not
    metrics_filled = not results_df.loc[(setup_name, slice(None), fine_tuning), model_name].isnull().any()
    return metrics_filled

def run_model(setup_name, tl_data):
    source_train = tl_data["source_train"]
    source_val = tl_data["source_validation"]
    target_fine_tuning = tl_data["target_fine_tuning"]
    target_test = tl_data["target_test"]
    target_train = tl_data["target_train"]

    for model_name, model_func in models.items():
        # check backwards if calculations have been done
        if is_metric_filled(setup_name, model_name, "full"):
            print(f"Skipping {model_name} for {setup_name} with full fine-tuning as metrics are already filled.")
            continue  # Skip this model if metrics are already filled
        
        # if not all fine-tuning scenarios are implemented
        # train model with source dataset
        model = model_func(source_train, source_val, epochs=TRAIN_EPOCHS)
        metrics = evaluate(model, target_test)

        # create deep copy for full fine-tuning
        full_training_model = copy.deepcopy(model)

        update_metrics(setup_name, model_name, "Zero-Shot", metrics['MAE'], metrics['MSE'])
        print(f"Metrics updated for {model_name} in {setup_name} with zero-shot fine-tuning: MAE = {metrics['MAE']}, MSE = {metrics['MSE']}")
        
        # fine tune on small target train set
        model = fine_tune_model(model, target_fine_tuning, epochs=TUNE_EPOCHS)
        metrics = evaluate(model, target_test)
        update_metrics(setup_name, model_name, "four_weeks", metrics['MAE'], metrics['MSE'])

        # fine tune on full target train set
        full_training_model = fine_tune_model(full_training_model, target_train, epochs=TUNE_EPOCHS)
        metrics = evaluate(full_training_model, target_test)
        update_metrics(setup_name, model_name, "full", metrics['MAE'], metrics['MSE'])

        # safe after every dataset combination
        results_df.to_csv(metrics_output_path)



# Execute for each setup and fine-tuning scenario
for setup_name, (source_data, target_data) in tl_setups.items():
    print(setup_name)
    tl_data = process_tl_data(source_data, target_data)
    run_model(setup_name, tl_data)

results_df


/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/py ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


elec_to_bavaria


`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 40.74it/s]


/tmp/ipykernel_7633/2506840223.py:24: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, "MAE", fine_tuning), model_name] = mae
/tmp/ipykernel_7633/2506840223.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, "MSE", fine_tuning), model_name] = mse
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rin           | RINorm           | 118   
4 | stacks        | ModuleList       | 13.0 M
---------------------------------------------------
11.6 M    Trainable params
1.5 M     Non-trainable params
13.0 

...

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 151/151 [00:01<00:00, 94.56it/s, train_loss=0.196]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:00<00:00, 30.87it/s]


NHiTS                     NBEATS  \
Setup           Metric Fine-Tuning                                          
elec_to_bavaria MSE    Zero-Shot      0.001494                   0.001622   
                       four_weeks      0.00038                   0.001545   
                       full           0.000285                   0.000341   
                MAE    Zero-Shot      0.027896                   0.029738   
                       four_weeks     0.010166                   0.028302   
                       full           0.009162                   0.010483   
elec_to_gp      MSE    Zero-Shot      0.978814         3129646596046398.5   
                       four_weeks      1.62771       331084261892769920.0   
                       full           0.701723      58956256895080.835938   
                MAE    Zero-Shot      0.625532            33161631.906993   
                       four_weeks     0.819922           328750785.919561   
                       full           0.486887             4643046.335466   
bavaria_to_elec MSE    Zero-Shot     23.056707                   8.605089   
                       four_weeks      0.78179                25837.52419   
                       full           0.339387                   1.153949   
                MAE    Zero-Shot      4.155268                   1.619315   
                       four_weeks     0.707144                 103.789648   
                       full           0.399113                   0.867067   
bavaria_to_gp   MSE    Zero-Shot    423.471861  30286062068186811990016.0   
                       four_weeks     4.731295      1117790530251497088.0   
                       full           0.708775      29681928207934.714844   
                MAE    Zero-Shot     13.512642         72424418016.978973   
                       four_weeks     1.401244           521314913.904936   
                       full           0.488889             3338412.520395   
gp_to_bavaria   MSE    Zero-Shot      0.001452                   0.001608   
                       four_weeks     0.000365                   0.001384   
                       full            0.00027                   0.000334   
                MAE    Zero-Shot      0.027271                     0.0295   
                       four_weeks     0.010501                   0.024949   
                       full            0.00897                   0.010353   
gp_to_elec      MSE    Zero-Shot      0.722685                   1.025007   
                       four_weeks     0.447453                   49.58384   
                       full           0.337812               28988.593879   
                MAE    Zero-Shot      0.662035                    0.82982   
                       four_weeks     0.484083                   4.558356   
                       full           0.397553                 126.873278   

                                   Transformer   TSMixer  
Setup           Metric Fine-Tuning                        
elec_to_bavaria MSE    Zero-Shot      0.001694  0.001299  
                       four_weeks     0.000609  0.000579  
                       full           0.000307  0.000313  
                MAE    Zero-Shot      0.029968  0.026338  
                       four_weeks     0.013808  0.012281  
                       full            0.00902  0.009167  
elec_to_gp      MSE    Zero-Shot      1.379277  1.143603  
                       four_weeks     0.969467  0.839688  
                       full           0.694619  0.620575  
                MAE    Zero-Shot      0.763253  0.661247  
                       four_weeks     0.602427  0.540042  
                       full           0.465008  0.422698  
bavaria_to_elec MSE    Zero-Shot      9.996558  11.72056  
                       four_weeks     0.988933  1.015349  
                       full           0.374176  0.269388  
                MAE    Zero-Shot      2.791272  3.066693  
                       four_weeks     0.821071  0.83